# 1. Setup

In [ ]:
!nvidia-smi

In [ ]:
%%capture
!pip install pandas
!pip install numpy
!pip install scikit-learn
!pip install warnings
!pip install catboost
!pip install openpyxl

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import f1_score, brier_score_loss
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

import warnings
warnings.filterwarnings("ignore")

SEED = 42
REPEATS = 5
FOLDS = 5

In [ ]:
train_combined = pd.read_csv("train_processed_ver2.csv").set_index('participant_id')   
test_combined = pd.read_csv("test_processed_ver2.csv").set_index('participant_id')  

labels = pd.read_excel("TRAINING_SOLUTIONS.xlsx").set_index("participant_id")

train_combined = train_combined.sort_index()
labels = labels.sort_index()

y_adhd = labels['ADHD_Outcome']
y_sex = labels['Sex_F']

combinations = y_adhd.astype(str) + y_sex.astype(str)
assert all(train_combined.index == labels.index), "Label IDs do not match train IDs"

def eval_metrics(y_true, y_pred, weights, label="None", threshold=0.6):
    brier = brier_score_loss(y_true, y_pred)
    f1 = f1_score(y_true, (y_pred > threshold).astype(int), sample_weight=weights)
    print(f"{label} -> Brier Score: {brier:.4f}, F1: {f1:.4f}")
    return brier, f1

In [ ]:
train_combined.columns

In [ ]:
numerical_cols = ['EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
       'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
       'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
       'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
       'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
       'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
       'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan',
]

categorical_cols = [col for col in train_combined.columns if col not in numerical_cols]

columns_to_scale = numerical_cols + ['Barratt_Barratt_P1_Edu', 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Edu', 'Barratt_Barratt_P2_Occ',]

ct = ColumnTransformer(
    transformers=[
        ('num_ord_scaler', StandardScaler(), columns_to_scale)
    ],
    remainder='passthrough'  # keep other columns (like one-hot) as-is
)

X_train = ct.fit_transform(train_combined)
X_test = ct.transform(test_combined)

In [ ]:
remaining_col_names = [col for col in train_combined.columns if col not in columns_to_scale]
final_col_names = columns_to_scale + remaining_col_names

train_combined = pd.DataFrame(X_train, columns=final_col_names, index=train_combined.index)
test_combined = pd.DataFrame(X_test, columns=final_col_names, index=test_combined.index)

# 2. Feature Selection

In [ ]:
# features_adhd = ['SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Prosocial', 'Basic_Demos_Enroll_Year_2017', 'Basic_Demos_Enroll_Year_2018', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_ID']

# interactions = ['SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Peer_Problems','Basic_Demos_Enroll_Year_2019', 'Basic_Demos_Enroll_Year_Other', 'APQ_P_APQ_P_ID']

# features_sex = ['SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Emotional_Problems', 'ColorVision_CV_Score', 'APQ_P_APQ_P_PP', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'SDQ_SDQ_Internalizing'] + ['adhd_proba'] + [f"I_{feat}" for feat in interactions]

In [ ]:
features_adhd = ['SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Generating_Impact', 'SDQ_SDQ_Conduct_Problems', 'SDQ_SDQ_Internalizing', 'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Emotional_Problems', 'SDQ_SDQ_Prosocial', 'Basic_Demos_Enroll_Year_2017', 'Basic_Demos_Enroll_Year_2018', 'APQ_P_APQ_P_OPD', 'APQ_P_APQ_P_ID']

features_sex = ['SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Prosocial', 'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Emotional_Problems', 'ColorVision_CV_Score', 'APQ_P_APQ_P_PP', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD', 'SDQ_SDQ_Internalizing'] 

In [ ]:
# Initialize out-of-fold arrays and scores
sex_oof = np.zeros(len(y_sex))
adhd_oof = np.zeros(len(y_adhd))
scores_sex = []
scores_adhd = []

t_sex = 0.3
t_adhd = 0.4

rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS)

# Updated parameters including class_weight to balance the classes based on their distribution
params = {
    "penalty": "l1",
    "Cs": 10,
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga",
    "class_weight": "balanced"   # This tells the model to adjust weights inversely proportional to class frequencies
}

model_adhd = LogisticRegressionCV(**params)
model_sex  = LogisticRegressionCV(**params)

for fold, (train_idx, val_idx) in enumerate(rskf.split(train_combined, combinations), 1):
    print(f"\n=== Fold {fold} ===")
    X_train, X_val = train_combined.iloc[train_idx].copy(), train_combined.iloc[val_idx].copy()
    y_train_adhd, y_val_adhd = y_adhd.iloc[train_idx], y_adhd.iloc[val_idx]
    y_train_sex, y_val_sex = y_sex.iloc[train_idx], y_sex.iloc[val_idx]
    
    # Sample weights: 2x weight for "11" (female ADHD) cases, per competition requirements
    weights_train = np.where(combinations.iloc[train_idx] == "11", 2, 1)
    weights = np.where(combinations.iloc[val_idx] == "11", 2, 1)
    
    # ----- First Stage: ADHD Model -----
    model_adhd.fit(X_train[features_adhd], y_train_adhd, sample_weight=weights_train)
    adhd_train = model_adhd.predict_proba(X_train[features_adhd])[:, 1]
    adhd_val   = model_adhd.predict_proba(X_val[features_adhd])[:, 1]
    adhd_oof[val_idx] += adhd_val / REPEATS
    adhd_brier, adhd_f1 = eval_metrics(y_val_adhd, adhd_val, weights, "ADHD", t_adhd)
    scores_adhd.append((adhd_brier, adhd_f1))

    model_sex.fit(X_train[features_sex], y_train_sex, sample_weight=weights_train)
    sex_val = model_sex.predict_proba(X_val[features_sex])[:, 1]
    sex_oof[val_idx] += sex_val / REPEATS
    sex_brier, sex_f1 = eval_metrics(y_val_sex, sex_val, weights, "Sex_F", t_sex)
    scores_sex.append((sex_brier, sex_f1))
    
print(f"\n=== CV Results ===")
print(f"Sex Mean Brier Score: {np.mean([s[0] for s in scores_sex]):.4f}")
print(f"Sex Mean F1: {np.mean([s[1] for s in scores_sex]):.4f}")
print(f"ADHD Mean Brier Score: {np.mean([s[0] for s in scores_adhd]):.4f}")
print(f"ADHD Mean F1: {np.mean([s[1] for s in scores_adhd]):.4f}")

# 3. Threshold Optimization

In [ ]:
weights = ((y_adhd == 1) & (y_sex == 1)) + 1
thresholds = np.linspace(0, 1, 100)

# ADHD threshold optimization
adhd_scores = [f1_score(y_adhd, (adhd_oof > t).astype(int), sample_weight=weights) for t in thresholds]
best_adhd_threshold = thresholds[np.argmax(adhd_scores)]
best_adhd_score = max(adhd_scores)

# Sex threshold optimization
sex_scores = [f1_score(y_sex, (sex_oof > t).astype(int), sample_weight=weights) for t in thresholds]
best_sex_threshold = thresholds[np.argmax(sex_scores)]
best_sex_score = max(sex_scores)

# 4. Inference

In [ ]:
model_adhd.fit(train_combined[features_adhd], y_adhd, sample_weight=weights)
adhd_proba_train = model_adhd.predict_proba(train_combined[features_adhd])[:, 1]
adhd_proba_test = model_adhd.predict_proba(test_combined[features_adhd])[:, 1]

train_combined["adhd_proba"] = adhd_proba_train
test_combined["adhd_proba"] = adhd_proba_test

model_sex.fit(train_combined[features_sex], y_sex, sample_weight=weights)
sex_proba_test = model_sex.predict_proba(test_combined[features_sex])[:, 1]

# Submission

In [ ]:
submission = pd.read_excel("data/SAMPLE_SUBMISSION.xlsx")
submission["ADHD_Outcome"] = np.where(adhd_proba_test > best_adhd_threshold, 1, 0)
submission["Sex_F"] = np.where(sex_proba_test > best_sex_threshold, 1, 0)
# Compare share of predicted labels at thresholds between OOF and Test
print(f"Share ADHD OOF: {np.mean(np.where(adhd_oof > best_adhd_threshold, 1, 0)):.4f} - Share ADHD Test: {submission.ADHD_Outcome.mean():.4f}")
print(f"Share Sex_F OOF: {np.mean(np.where(sex_oof > best_sex_threshold, 1, 0)):.4f} - Share Sex_F Test: {submission.Sex_F.mean():.4f}")

submission.to_csv("submission_ver2.csv", index=False)

Version 1
+ Share ADHD OOF: 0.8384 - Share ADHD Test: 0.8454
+ Share Sex_F OOF: 0.9563 - Share Sex_F Test: 0.9013